In [1]:
import deepchem as dc
import rdkit
from rdkit import Chem
import numpy as np
import pandas as pd
import scipy.sparse as sp
import pickle

import os
from constants import *
import warnings
warnings.filterwarnings('ignore')

2024-11-06 20:39:57.578385: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 20:40:01.120995: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ohpc/pub/mpi/openmpi3-gnu7/3.1.0/lib:/opt/ohpc/pub/compiler/gcc/7.3.0/lib64
2024-11-06 20:40:01.121086: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ohpc/pub/mpi/openmpi3-gnu7/3.1.0/lib:/opt/oh

In [2]:
global suffix

In [3]:
def get_csv(path, smipos):
    df = pd.read_pickle(path)
    df = df.rename(columns={smipos: "SMILES_NS"})

    return df


def preprocess_graph(
    data,
):  # The function is to preprocessed the adjacency matrix, returning the normalized adjacency matrix in the form of numpy array for feeding into the model
    adj_ = data + sp.eye(data.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = np.diag(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt)
    return np.array(adj_normalized)


def smiles_get_features(
    a,
):  # This function will return the smiles code into list of feature for each atoms
    if isinstance(a, float):
        return np.nan
    m = rdkit.Chem.MolFromSmiles(a)
    featurizer = dc.feat.ConvMolFeaturizer()
    features = featurizer.featurize([m])[0]
    if isinstance(features, np.ndarray):
        return np.nan
    atom_features = features.get_atom_features()  # initial atom feature vectors
    if atom_features.shape[0] > 60:
        return np.nan
    return atom_features

In [4]:
def smiles_get_adj(a):  # This function retrieve the adjacency matrix from the molecule
    if isinstance(a, float):
        return np.nan
    m = rdkit.Chem.MolFromSmiles(a)
    featurizer = dc.feat.ConvMolFeaturizer()
    features = featurizer.featurize([m])[0]
    if isinstance(features, np.ndarray):
        return np.nan
    adj_list = features.get_adjacency_list()  # adjacency list (neighbor list)
    adj = np.zeros(
        (len(adj_list), len(adj_list))
    )  # convert adjacency list into adjacency matrix "A"
    if len(adj_list) > 60:
        return np.nan
    for i in range(len(adj_list)):
        for j in adj_list[i]:
            adj[i][j] = 1
    return adj


def smiles_get_edge(a):  # This function retrieve the adjacency matrix from the molecule
    if isinstance(a, float):
        return np.nan
    m = rdkit.Chem.MolFromSmiles(a)
    featurizer = dc.feat.ConvMolFeaturizer()
    features = featurizer.featurize([m])[0]
    if isinstance(features, np.ndarray):
        return np.nan
    adj_list = features.get_adjacency_list()  # adjacency list (neighbor list)
    node1 = []
    node2 = []
    for i in range(len(adj_list)):
        for j in adj_list[i]:
            node1.append(i)
            node2.append(j)
    return np.stack((np.array(node1), np.array(node2)))


def sim_graph(smile):
    if isinstance(smile, float):
        return np.nan
    mol = rdkit.Chem.MolFromSmiles(smile)
    if mol is None:
        return np.nan
    Chem.Kekulize(mol)
    atoms = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    am = Chem.GetAdjacencyMatrix(mol, useBO=True)
    if len(atoms) > 60:
        return np.nan
    for i, atom in enumerate(atoms):
        am[i, i] = atom
    return am

In [5]:
def get_max_dim(
    d,
):  # This function is used to find the maximum dimension the set of data contain
    maxdim = 0
    for i in d:
        if i.shape[0] > maxdim:
            maxdim = i.shape[0]
    return maxdim


def pad_up_to(
    t, max_in_dims, constant_values=0
):  # This function is used to pad the data up to a given dimension
    s = t.shape
    size = np.subtract(max_in_dims, s)
    return np.pad(
        t, ((0, size[0]), (0, size[1])), "constant", constant_values=constant_values
    )


def get_np_adj_label(
    df,
    path,
    smilepos,
    labelpos,
    fingerpos,
    fingername,
    idpos,
    pad_dim=None,
    save=True,
    Finger=False,
):
    smi = df[smilepos]
    prelabel = df[labelpos]
    idx = df[idpos]
    max_dim = 60

    print("Getting numpy files for", len(smi), "smiles")
    # print(df,smilepos)

    if Finger:
        fing = df.iloc[:, fingerpos:]
    smi_to_adj = {}
    smi_to_edge = {}
    padded_adj_map = {}
    for smiles in smi.unique():
        pre_adj_i = smiles_get_adj(smiles)
        smi_to_adj[smiles] = pre_adj_i
        smi_to_edge[smiles] = smiles_get_edge(smiles)
        if pre_adj_i is not np.nan:
            padded_adj_map[smiles] = pad_up_to(preprocess_graph(pre_adj_i), max_dim, max_dim)
        
    pre_adj = smi.map(smi_to_adj)
    edge = smi.map(smi_to_edge).rename("edge")
    if Finger:
        adj_label = pd.concat([pre_adj, prelabel, edge, fing], axis=1, sort=False)
    else:
        adj_label = pd.concat([idx, pre_adj, edge, prelabel], axis=1, sort=False)
    adj_label = adj_label[adj_label[smilepos].notna()]
    Trueedge = list(adj_label["edge"].values)
    Truelabel = adj_label[labelpos]
    Trueidx = adj_label[idpos]
    print("ID pos:", idpos)
    if Finger:
        Fingerprint = adj_label.iloc[:, fingerpos:].values
    
    if save:
        print("Saving...")
        if Finger:
            np.save(
                path + "/" + fingername + "_fingerprint", Fingerprint, fix_imports=False
            )
        np.save(path + f"/label{suffix}", Truelabel, fix_imports=False)
        print(Trueidx)
        np.save(path + f"/idx{suffix}", Trueidx, fix_imports=False)
        with open(path + f"/edge{suffix}", "wb") as f:
            pickle.dump(Trueedge, f)
        print("Saved")
    
    print("Preprocessing graph...")
    # True_adj = pre_adj.apply(preprocess_graph)
        
    True_adj = smi.map(padded_adj_map).dropna()
    True_array_ADJ = np.stack(True_adj.values)
    print("Done")      
    if save:
        print("Saving...")
        np.save(path + f"/adj{suffix}", True_array_ADJ, fix_imports=False)
        print("Saved")
    if Finger:
        return [True_array_ADJ, max_dim, Truelabel, Fingerprint]
    else:
        return [True_array_ADJ, Trueedge, max_dim, Truelabel]

In [6]:
def get_feature(df, max_dim, smilepos, path, save=True):
    smi = df[smilepos]
    print("Getting features for", len(smi), "smiles")
    smi_unique = pd.Series(smi.unique())
    true_feature_map = dict(zip(smi_unique.tolist(), smi_unique
                              .apply(smiles_get_features)
                              .apply(lambda x: np.nan 
                                     if x is np.nan 
                                     else pad_up_to(x, (max_dim, 75)))
                              .tolist()))
                        
    # pre_feature = smi.apply(smiles_get_features).dropna()
    # True_feature = pre_feature.apply(pad_up_to, args=((max_dim, 75),))
    True_feature = smi.map(true_feature_map).dropna()
    True_array_feature = np.stack(True_feature.values)
    if save:
        np.save(path + f"/feature{suffix}", True_array_feature, fix_imports=False)
    return True_array_feature


def get_graph(df, max_dim, smilepos, path, save=True):
    smi = df[smilepos]
    print("Getting graph files for", len(smi), "smiles")
    smi_unique = pd.Series(smi.unique())
    true_graph_map = dict(zip(smi_unique.tolist(), smi_unique
                              .apply(sim_graph)
                              .apply(lambda x: np.nan 
                                     if x is np.nan 
                                     else pad_up_to(x, (max_dim, max_dim)))
                              .tolist()))
    true_graph = smi.map(true_graph_map).dropna()
    # true_graph = pre_graph.apply(pad_up_to, args=((max_dim, max_dim),))
    true_array_graph = np.stack(true_graph.values)
    if save:
        np.save(path + f"/graph{suffix}", true_array_graph, fix_imports=False)
    return true_array_graph


def get_smiles(df, max_dim, smilepos, path, save=True):
    smi = df[smilepos]
    if save:
        np.save(path + f"/smiles{suffix}", smi, fix_imports=False)
    return smi


class GraphGenerator:
    def __init__(self, datapath, nppath):
        os.makedirs(nppath, exist_ok=True)
        self.df = get_csv(datapath, f"molecule_structures{suffix}")
        self.save = True
        self.smilepos = "SMILES_NS"
        self.idpos = "id"
        self.fingername = "MK"
        self.labelpos = ["synergy_loewe"]
        self.fingerpos = 2

        self.adj, self.edge, self.max_dim, self.label = get_np_adj_label(
            self.df,
            nppath,
            self.smilepos,
            self.labelpos,
            self.fingerpos,
            self.fingername,
            self.idpos,
            save=self.save,
            Finger=False,
        )

        self.feature = get_feature(
            self.df, self.max_dim, self.smilepos, nppath, save=self.save
        )
        self.graph = get_graph(
            self.df, self.max_dim, self.smilepos, nppath, save=self.save
        )
        self.smiles = get_smiles(
            self.df, self.max_dim, self.smilepos, nppath, save=self.save
        )

In [7]:
suffix = "_row"
graph_generator = GraphGenerator(
    f"{EXPORT_DATA_PATH}/data_drugcomb.pkl", f"{DATA_PATH}/nps"
)

Getting numpy files for 131790 smiles
ID pos: id
Saving...
0              0
1              1
2              2
3              3
4              4
           ...  
131782    131782
131784    131784
131786    131786
131787    131787
131789    131789
Name: id, Length: 125241, dtype: int64
Saved
Preprocessing graph...
Done
Saving...
Saved
Getting features for 131790 smiles
Getting graph files for 131790 smiles


In [8]:
x = get_csv(f"{EXPORT_DATA_PATH}/data_drugcomb.pkl", "molecule_structures_col")
np.unique(x["SMILES_NS"].notna())

array([ True])

In [9]:
# All three should have the same size
np_label = np.load(f"{DATA_PATH}/nps/idx_col.npy")
print(len(np_label))
np_label = np.load(f"{DATA_PATH}/nps/label_col.npy")
print(len(np_label))
np_label = np.load(f"{DATA_PATH}/nps/adj_col.npy")
print(len(np_label))

125857
125857
125857


In [10]:
# Check intersection of row and col indices
col_idx = np.load(f"{DATA_PATH}/nps/idx_col.npy")
row_idx = np.load(f"{DATA_PATH}/nps/idx_row.npy")
all_idx = np.intersect1d(col_idx, row_idx)
len(all_idx)

119600

In [11]:
np.save(f"{DATA_PATH}/nps/idx_intersect", all_idx, fix_imports=False)

In [12]:
col_idx = np.load(f"{DATA_PATH}/nps/idx_col.npy")
col_idx = [idx for idx, x in enumerate(col_idx) if x in all_idx]
col_label = np.load(f"{DATA_PATH}/nps/label_col.npy")
len(col_label[col_idx])

119600

In [13]:
row_idx = np.load(f"{DATA_PATH}/nps/idx_row.npy")
row_idx = [idx for idx, x in enumerate(row_idx) if x in all_idx]
row_label = np.load(f"{DATA_PATH}/nps/label_row.npy")
row_label = row_label[row_idx]
row_adj = np.load(f"{DATA_PATH}/nps/adj_row.npy")
row_adj = row_adj[row_idx]
row_smiles = np.load(f"{DATA_PATH}/nps/smiles_row.npy", allow_pickle=True)
row_smiles = row_smiles[row_idx]

np.save(f"{DATA_PATH}/nps_intersected/idx_row", row_idx)
np.save(f"{DATA_PATH}/nps_intersected/label_row", row_label)
np.save(f"{DATA_PATH}/nps_intersected/adj_row", row_adj)
np.save(f"{DATA_PATH}/nps_intersected/smiles_row", row_smiles)

In [14]:
col_idx = np.load(f"{DATA_PATH}/nps/idx_col.npy")
col_idx = [idx for idx, x in enumerate(col_idx) if x in all_idx]
col_label = np.load(f"{DATA_PATH}/nps/label_col.npy")
col_label = col_label[col_idx]
col_adj = np.load(f"{DATA_PATH}/nps/adj_col.npy")
col_adj = col_adj[col_idx]
col_smiles = np.load(f"{DATA_PATH}/nps/smiles_col.npy", allow_pickle=True)
col_smiles = col_smiles[col_idx]

np.save(f"{DATA_PATH}/nps_intersected/col_row", col_idx)
np.save(f"{DATA_PATH}/nps_intersected/label_col", col_label)
np.save(f"{DATA_PATH}/nps_intersected/adj_col", col_adj)
np.save(f"{DATA_PATH}/nps_intersected/smiles_col", col_smiles)